En este Notebook, se presenta un conjunto de pasos para realizar el proceso de Extracción, Carga y Transformación del cojunto de datos a utilizar en el MVP (Minimum Viable Product) o Producto Mínimo Viable

In [35]:
import pandas as pd
import numpy as np
import ast
import gzip
import re

In [36]:
# Carga del Dataset users_items.json
ruta_archivo = '..\\Datasets\\users_items.json.gz'

json_datos = list()

# Abrir y leer el archivo users_items.json.gz
with gzip.open(ruta_archivo, 'rt', encoding='utf-8') as archivo_json:
    for linea in archivo_json:
        try:
            # Evaluar una cadena que representa un diccionario
            json_data = ast.literal_eval(linea)
            json_datos.append(json_data)
        except ValueError as e:
            print(f"ERROR al leer o extraer : {linea}")
            continue

In [37]:
# Transformar la lista json_datos a un dataframe
df_items = pd.DataFrame(json_datos)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [38]:
# Verificar dimensiones del dataframe (Filas, Columnas)
print(f'Filas : {df_items.shape[0]}, Columnas : {df_items.shape[1]}')

Filas : 88310, Columnas : 5


In [39]:
# Verificar la cantidad de nulos que pude tener el dataframe 
df_items.isna().sum()

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

In [40]:
# Observamos dos variables que no nos aporta información para nuestras consignas y la cual seran ecluidas del dataframe
variables_excluidas = ['steam_id', 'user_url']
df_items.drop(columns= variables_excluidas, inplace= True)

In [41]:
df_items.head(3)

,user_id,items_count,items
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [42]:
# Transformar el dataFrame, a traves de los valores de "items" en que cada elemento de la lista de una fila independiente
df_items = df_items.explode('items').reset_index(drop=True)
items = pd.json_normalize(df_items['items']).reset_index(drop=True)
df_items = pd.concat([df_items.drop('items', axis=1), items], axis=1)
df_items.shape

(5170015, 6)

In [43]:
df_items.columns

Index(['user_id', 'items_count', 'item_id', 'item_name', 'playtime_forever',
       'playtime_2weeks'],
      dtype='object')

In [44]:
# Observamos un par de variables que no nos aporta informacion, las vamos a exluir del dataframe
variables_excluidas = ['item_id','item_name']
df_items.drop(columns = variables_excluidas, inplace=True)
df_items.head(3)

,user_id,items_count,playtime_forever,playtime_2weeks
0,76561197970982479,277,6.0,0.0
1,76561197970982479,277,0.0,0.0
2,76561197970982479,277,7.0,0.0


In [45]:
# Verificar la existencia de duplicados que nos permita reducir el numero de lineas
df_items.duplicated().sum()

2253629

In [46]:
# Eliminar duplicados puede tomar la opcion keep='first' o keep='last' mantener el primero o el ultimo de las instancia 
df_items.drop_duplicates(keep='first',inplace=True)
df_items.shape

(2916386, 4)

In [47]:
# Crear un archivo con formato parquet
ruta_archivo = '..\\Datasets\\users_items.parquet'
df_items.to_parquet(ruta_archivo)